<a href="https://colab.research.google.com/github/gadmin7/EventExtraction/blob/main/eventExtractionOnRAMS_arguments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q evaluate
!pip install -q seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 904.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 890.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import json
import evaluate
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict, ClassLabel,Features
import datasets
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data = []
with open(r"/content/drive/MyDrive/NLP_event_extraction/train.jsonlines",encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
print(data[0])

{'rel_triggers': [], 'gold_rel_links': [], 'doc_key': 'nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3a0910dca8b5b62f', 'ent_spans': [[42, 43, [['evt090arg02victim', 1.0]]], [85, 88, [['evt090arg01killer', 1.0]]], [26, 26, [['evt090arg04place', 1.0]]]], 'language_id': 'eng', 'source_url': 'https://www.washingtonpost.com/news/powerpost/paloma/daily-202/2016/06/17/daily-202-more-republicans-ditch-trump-conclude-he-cannot-win/5763a1e0981b92a22d0f8a36/', 'evt_triggers': [[69, 69, [['life.die.deathcausedbyviolentevents', 1.0]]]], 'split': 'train', 'sentences': [['Transportation', 'officials', 'are', 'urging', 'carpool', 'and', 'teleworking', 'as', 'options', 'to', 'combat', 'an', 'expected', 'flood', 'of', 'drivers', 'on', 'the', 'road', '.'], ['(', 'Paul', 'Duggan', ')'], ['--', 'A', 'Baltimore', 'prosecutor', 'accused', 'a', 'police', 'detective', 'of', '“', 'sabotaging', '”', 'investigations', 'related', 'to', 'the', 'death', 'of', 'Freddie', 'Gray', ',', 'accusing', 'him', 'of', 'fabrica

In [ ]:
data_test = []
with open(r"/content/drive/MyDrive/NLP_event_extraction/dev.jsonlines",encoding='utf-8') as f:
    for line in f:
        data_test.append(json.loads(line))

In [ ]:
df1 = pd.DataFrame(data_test)
df1[['doc_key','sentences','gold_evt_links']].head(6)

,doc_key,sentences,gold_evt_links
0,nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,"[[Three, specific, points, illustrate, why, Am...","[[[40, 40], [33, 33], evt089arg01victim], [[40..."
1,nw_RC010f29c9438b939daa050b70d8127eb5b2b3cb927...,"[[The, fixture, ,, anticipated, by, fans, as, ...","[[[65, 65], [64, 64], evt011arg01attacker], [[..."
2,nw_RC013008ad72d04b5e4cca4706cad5cc71c88b0df46...,"[[In, addition, to, working, alongside, super,...",[]
3,nw_RC0136ad4663cedfb31bbf25548fc42bf20b1bb67c8...,"[[An, estimated, 60,000, to, 70,000, were, kil...","[[[47, 47], [45, 45], evt018arg01yielder]]"
4,nw_RC01bd87a7362ff76837d24872455f0e00f7056f18f...,"[[His, campaign, has, just, announced, it, wil...","[[[38, 38], [34, 35], evt139arg01giver], [[38,..."
5,nw_RC01e3d033aaae228234b285e4ab1e92c3291039594...,"[[Ted, Cruz], [Ted, Cruz, took, swipes, at, Tr...","[[[26, 26], [18, 19], evt061arg01participant],..."


In [ ]:
df = pd.DataFrame(data)
df.head(2)

,rel_triggers,gold_rel_links,doc_key,ent_spans,language_id,source_url,evt_triggers,split,sentences,gold_evt_links
0,[],[],nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,"[[42, 43, [['evt090arg02victim', 1.0]]], [85, ...",eng,https://www.washingtonpost.com/news/powerpost/...,"[[69, 69, [['life.die.deathcausedbyviolenteven...",train,"[[Transportation, officials, are, urging, carp...","[[[69, 69], [85, 88], evt090arg01killer], [[69..."
1,[],[],nw_RC0006850e77f498cb669777ce71e4ebfc471911174...,"[[83, 85, [['evt105arg05origin', 1.0]]], [26, ...",eng,http://www.expressen.se/nyheter/womens-secret-...,"[[51, 51, [['movement.transportartifact.hide',...",train,"[[2014, :, ISIS, marches, into, Raqqa, and, ma...","[[[51, 51], [26, 30], evt105arg01transporter],..."


In [ ]:
df1.drop(['rel_triggers','gold_rel_links','language_id','source_url','split'],axis=1,inplace=True)

In [ ]:
df.drop(['rel_triggers','gold_rel_links','language_id','source_url','split'],axis=1,inplace=True)

In [ ]:
%%time
dataframe_list = []
for idx in range(7329):

    row_tokens = []
    trigger_tags = []
    ner_tags = []
    arg_tags = []
    doc_key = []
    for i in range(len(df['sentences'][idx])):
        row_tokens.extend(df['sentences'][idx][i])

    for i in range(len(row_tokens)):
        trigger_tags.append('O')
        arg_tags.append('O')
        ner_tags.append('O')
        doc_key.append(df['doc_key'][idx])

    gold_link = df['gold_evt_links'][idx]

    for i in range(len(gold_link)):

        trigger_list = gold_link[i][0]
        arg_list = gold_link[i][1]
        arg_label = gold_link[i][2]

        trigger_beg = trigger_list[0]
        trigger_end = trigger_list[1]
        trigger_tags[trigger_beg] = 'B-'+ arg_label[:6]
        if trigger_beg != trigger_end:
            for trg in range(trigger_beg+1,trigger_end+1):
                trigger_tags[trg] = 'I-'+ arg_label[:6]

        arg_beg = arg_list[0]
        arg_end = arg_list[1]
        arg_tags[arg_beg] = 'B-'+ arg_label[6:11]
        if arg_beg != arg_end:
            for arg in range(arg_beg+1,arg_end+1):
                arg_tags[arg] = 'I-'+ arg_label[6:11]

        arg_beg = arg_list[0]
        arg_end = arg_list[1]
        ner_tags[arg_beg] = 'B-'+ arg_label[11:]
        if arg_beg != arg_end:
            for arg in range(arg_beg+1,arg_end+1):
                ner_tags[arg] = 'I-'+ arg_label[11:]

    d1 = pd.DataFrame({'doc_key':doc_key,'token':row_tokens,'event_tags':trigger_tags,'arg_tags':arg_tags,'ner_tags':ner_tags})
    dataframe_list.append(d1)

CPU times: user 11 s, sys: 222 ms, total: 11.3 s
Wall time: 11.2 s


In [ ]:
%%time
dataframe_test_list = []
for idx in range(df1.shape[0]):

    row_tokens = []
    trigger_tags = []
    arg_tags = []
    ner_tags = []
    doc_key = []
    for i in range(len(df1['sentences'][idx])):
        row_tokens.extend(df1['sentences'][idx][i])

    for i in range(len(row_tokens)):
        trigger_tags.append('O')
        arg_tags.append('O')
        ner_tags.append('O')
        doc_key.append(df1['doc_key'][idx])

    gold_link = df1['gold_evt_links'][idx]

    for i in range(len(gold_link)):

        trigger_list = gold_link[i][0]
        arg_list = gold_link[i][1]
        arg_label = gold_link[i][2]

        trigger_beg = trigger_list[0]
        trigger_end = trigger_list[1]
        trigger_tags[trigger_beg] = 'B-'+ arg_label[:6]
        if trigger_beg != trigger_end:
            for trg in range(trigger_beg+1,trigger_end+1):
                trigger_tags[trg] = 'I-'+ arg_label[:6]

        arg_beg = arg_list[0]
        arg_end = arg_list[1]
        arg_tags[arg_beg] = 'B-'+ arg_label[6:11]
        if arg_beg != arg_end:
            for arg in range(arg_beg+1,arg_end+1):
                arg_tags[arg] = 'I-'+ arg_label[6:11]

        arg_beg = arg_list[0]
        arg_end = arg_list[1]
        ner_tags[arg_beg] = 'B-'+ arg_label[11:]
        if arg_beg != arg_end:
            for arg in range(arg_beg+1,arg_end+1):
                ner_tags[arg] = 'I-'+ arg_label[11:]


    d1 = pd.DataFrame({'doc_key':doc_key,'token':row_tokens,'event_tags':trigger_tags,'arg_tags':arg_tags,'ner_tags':ner_tags})
    dataframe_test_list.append(d1)

CPU times: user 1.42 s, sys: 20.7 ms, total: 1.45 s
Wall time: 1.48 s


In [ ]:
data = pd.concat(dataframe_list)

In [ ]:
data[25:45]

,doc_key,token,event_tags,arg_tags,ner_tags
25,nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,A,O,O,O
26,nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,Baltimore,O,B-arg04,B-place
27,nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,prosecutor,O,O,O
28,nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,accused,O,O,O
29,nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,a,O,O,O
30,nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,police,O,O,O
31,nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,detective,O,O,O
32,nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,of,O,O,O
33,nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,“,O,O,O
34,nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,sabotaging,O,O,O


In [ ]:
test = pd.concat(dataframe_test_list)

In [ ]:
test[:50]

,doc_key,token,event_tags,arg_tags,ner_tags
0,nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,Three,O,O,O
1,nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,specific,O,O,O
2,nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,points,O,O,O
3,nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,illustrate,O,O,O
4,nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,why,O,O,O
5,nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,Americans,O,O,O
6,nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,see,O,O,O
7,nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,Trump,O,O,O
8,nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,as,O,O,O
9,nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,the,O,O,O


In [ ]:
def create_dataset(data):
    grouped_tokens = data.groupby('doc_key')['token'].apply(list).reset_index(name='token')
    grouped_events = data.groupby('doc_key')['event_tags'].apply(list).reset_index(name='event_tags')
    grouped_args = data.groupby('doc_key')['arg_tags'].apply(list).reset_index(name='arg_tags')
    grouped_ners = data.groupby('doc_key')['ner_tags'].apply(list).reset_index(name='ner_tags')

    dataset_dict = {
        "id": grouped_tokens['doc_key'],
        "tokens": grouped_tokens['token'],
        "event_tags": grouped_events['event_tags'],
        "arg_tags": grouped_args['arg_tags'],
        "ner_tags":grouped_ners['ner_tags']
    }

    return Dataset.from_dict(dataset_dict)

dataset1 = create_dataset(data)

In [ ]:
dataset2 = create_dataset(test)
dataset_dict = DatasetDict({"train": dataset1,"test":dataset2})

In [ ]:
dataset_dict['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'event_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'arg_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [ ]:
dataset_dict['test'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'event_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'arg_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [ ]:
print(dataset_dict['train'][0])

{'id': 'nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3a0910dca8b5b62f', 'tokens': ['Transportation', 'officials', 'are', 'urging', 'carpool', 'and', 'teleworking', 'as', 'options', 'to', 'combat', 'an', 'expected', 'flood', 'of', 'drivers', 'on', 'the', 'road', '.', '(', 'Paul', 'Duggan', ')', '--', 'A', 'Baltimore', 'prosecutor', 'accused', 'a', 'police', 'detective', 'of', '“', 'sabotaging', '”', 'investigations', 'related', 'to', 'the', 'death', 'of', 'Freddie', 'Gray', ',', 'accusing', 'him', 'of', 'fabricating', 'notes', 'to', 'suggest', 'that', 'the', 'state', '’s', 'medical', 'examiner', 'believed', 'the', 'manner', 'of', 'death', 'was', 'an', 'accident', 'rather', 'than', 'a', 'homicide', '.', 'The', 'heated', 'exchange', 'came', 'in', 'the', 'chaotic', 'sixth', 'day', 'of', 'the', 'trial', 'of', 'Baltimore', 'Officer', 'Caesar', 'Goodson', 'Jr.', ',', 'who', 'drove', 'the', 'police', 'van', 'in', 'which', 'Gray', 'suffered', 'a', 'fatal', 'spine', 'injury', 'in', '2015', '.', 

In [ ]:
arg_labels = ['O', 'B-arg04', 'B-arg02', 'I-arg02', 'B-arg01', 'I-arg01', 'B-arg05', 'I-arg05', 'I-arg04', 'B-arg03', 'I-arg03']

In [ ]:
event_labels = ['B-evt001', 'B-evt002', 'B-evt003', 'B-evt004', 'B-evt005', 'B-evt006', 'B-evt007', 'B-evt008', 'B-evt009', 'B-evt010', 'B-evt011', 'B-evt012', 'B-evt013', 'B-evt014', 'B-evt015', 'B-evt016', 'B-evt017', 'B-evt018',
                                     'B-evt019', 'B-evt020', 'B-evt021', 'B-evt022', 'B-evt023', 'B-evt024', 'B-evt025', 'B-evt026', 'B-evt027', 'B-evt028', 'B-evt029', 'B-evt030', 'B-evt031', 'B-evt032', 'B-evt033', 'B-evt034', 'B-evt035', 'B-evt036', 'B-evt037', 'B-evt038', 'B-evt039', 'B-evt040', 'B-evt041', 'B-evt042', 'B-evt043', 'B-evt044', 'B-evt045', 'B-evt046', 'B-evt047', 'B-evt048', 'B-evt049', 'B-evt050', 'B-evt051', 'B-evt052', 'B-evt053', 'B-evt054', 'B-evt055', 'B-evt057', 'B-evt059', 'B-evt060', 'B-evt061', 'B-evt062', 'B-evt063', 'B-evt064', 'B-evt065', 'B-evt066', 'B-evt068', 'B-evt070', 'B-evt071', 'B-evt072', 'B-evt073', 'B-evt074', 'B-evt075', 'B-evt076', 'B-evt077', 'B-evt079', 'B-evt080', 'B-evt081', 'B-evt082', 'B-evt083', 'B-evt084', 'B-evt085', 'B-evt086', 'B-evt087', 'B-evt088', 'B-evt089', 'B-evt090', 'B-evt091',
                                     'B-evt092', 'B-evt093', 'B-evt094', 'B-evt095', 'B-evt096', 'B-evt097', 'B-evt098', 'B-evt099', 'B-evt100', 'B-evt101', 'B-evt102', 'B-evt103', 'B-evt104', 'B-evt105', 'B-evt106', 'B-evt107', 'B-evt108', 'B-evt109', 'B-evt110', 'B-evt111', 'B-evt112', 'B-evt113', 'B-evt114', 'B-evt115', 'B-evt116', 'B-evt117', 'B-evt118', 'B-evt119', 'B-evt120', 'B-evt121', 'B-evt122', 'B-evt123', 'B-evt124', 'B-evt125', 'B-evt126', 'B-evt127', 'B-evt128', 'B-evt129', 'B-evt130', 'B-evt131', 'B-evt132', 'B-evt133', 'B-evt134', 'B-evt135', 'B-evt136', 'B-evt137', 'B-evt138', 'B-evt139', 'B-evt140', 'B-evt141', 'B-evt142', 'B-evt143', 'B-evt144', 'I-evt064', 'I-evt065', 'I-evt068', 'I-evt071', 'I-evt073', 'I-evt085', 'I-evt087', 'I-evt089', 'I-evt091', 'I-evt095', 'I-evt100', 'I-evt104', 'I-evt107', 'I-evt112', 'I-evt119', 'I-evt123', 'I-evt124', 'I-evt130', 'I-evt132', 'I-evt133', 'I-evt136', 'I-evt139', 'I-evt142', 'I-evt144', 'O']

In [ ]:
ner_labels = ['B-artifact', 'B-artifactmoney', 'B-attacker', 'B-ballot', 'B-beneficiary', 'B-candidate', 'B-communicator', 'B-crashobject', 'B-crime', 'B-damager', 'B-damagerdestroyer', 'B-deceased', 'B-defendant', 'B-demonstrator', 'B-destination', 'B-destroyer', 'B-detainee', 'B-driverpassenger', 'B-employee',
                                   'B-executioner', 'B-extraditer', 'B-fireexplosionobject', 'B-founder', 'B-giver', 'B-governmentbody', 'B-gpe', 'B-granter', 'B-hidingplace', 'B-injurer', 'B-inspectedentity', 'B-inspector', 'B-instrument', 'B-investigator', 'B-jailer', 'B-judgecourt', 'B-killer', 'B-law', 'B-manufacturer', 'B-money', 'B-monitor', 'B-monitoredentity', 'B-observedentity', 'B-observer', 'B-origin', 'B-otherparticipant', 'B-participant', 'B-passenger', 'B-place', 'B-placeofemployment', 'B-preventer', 'B-prosecutor', 'B-recipient', 'B-rejecternullifier', 'B-result', 'B-retreater', 'B-spy', 'B-surrenderer', 'B-target', 'B-territoryorfacility', 'B-transporter', 'B-vehicle', 'B-victim', 'B-violator',
                                   'B-voter', 'B-yielder', 'I-artifact', 'I-artifactmoney', 'I-attacker', 'I-ballot', 'I-beneficiary', 'I-candidate', 'I-communicator', 'I-crashobject', 'I-crime', 'I-damager', 'I-damagerdestroyer', 'I-deceased', 'I-defendant', 'I-demonstrator', 'I-destination', 'I-destroyer', 'I-detainee', 'I-driverpassenger', 'I-employee', 'I-executioner', 'I-extraditer', 'I-fireexplosionobject', 'I-founder', 'I-giver', 'I-governmentbody', 'I-gpe', 'I-granter', 'I-hidingplace', 'I-injurer', 'I-inspectedentity', 'I-inspector', 'I-instrument', 'I-investigator', 'I-jailer', 'I-judgecourt', 'I-killer', 'I-law', 'I-manufacturer', 'I-money', 'I-monitor', 'I-monitoredentity', 'I-observedentity', 'I-observer', 'I-origin',
                                   'I-otherparticipant', 'I-participant', 'I-passenger', 'I-place', 'I-placeofemployment', 'I-preventer', 'I-prosecutor', 'I-recipient', 'I-rejecternullifier', 'I-result', 'I-retreater', 'I-spy', 'I-surrenderer', 'I-target', 'I-territoryorfacility',
                                   'I-transporter', 'I-vehicle', 'I-victim', 'I-violator', 'I-voter', 'I-yielder', 'O']

In [ ]:
dataset_dict = dataset_dict.cast_column("event_tags", datasets.Sequence(ClassLabel(names=event_labels)))
dataset_dict = dataset_dict.cast_column("arg_tags", datasets.Sequence(ClassLabel(names=arg_labels)))

Casting the dataset:   0%|          | 0/7329 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/924 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7329 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/924 [00:00<?, ? examples/s]

In [ ]:
dataset_dict['test'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'event_tags': Sequence(feature=ClassLabel(names=['B-evt001', 'B-evt002', 'B-evt003', 'B-evt004', 'B-evt005', 'B-evt006', 'B-evt007', 'B-evt008', 'B-evt009', 'B-evt010', 'B-evt011', 'B-evt012', 'B-evt013', 'B-evt014', 'B-evt015', 'B-evt016', 'B-evt017', 'B-evt018', 'B-evt019', 'B-evt020', 'B-evt021', 'B-evt022', 'B-evt023', 'B-evt024', 'B-evt025', 'B-evt026', 'B-evt027', 'B-evt028', 'B-evt029', 'B-evt030', 'B-evt031', 'B-evt032', 'B-evt033', 'B-evt034', 'B-evt035', 'B-evt036', 'B-evt037', 'B-evt038', 'B-evt039', 'B-evt040', 'B-evt041', 'B-evt042', 'B-evt043', 'B-evt044', 'B-evt045', 'B-evt046', 'B-evt047', 'B-evt048', 'B-evt049', 'B-evt050', 'B-evt051', 'B-evt052', 'B-evt053', 'B-evt054', 'B-evt055', 'B-evt057', 'B-evt059', 'B-evt060', 'B-evt061', 'B-evt062', 'B-evt063', 'B-evt064', 'B-evt065', 'B-evt066', 'B-evt068', 'B-evt070', 'B-evt071', 'B-evt072', 'B-evt073', 'B

In [ ]:
label_list = dataset_dict["train"].features["arg_tags"].feature.names

In [ ]:
print(label_list)

['O', 'B-arg04', 'B-arg02', 'I-arg02', 'B-arg01', 'I-arg01', 'B-arg05', 'I-arg05', 'I-arg04', 'B-arg03', 'I-arg03']


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased",max_length=512,padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
example = dataset_dict["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens[:10]

['[CLS]',
 'transportation',
 'officials',
 'are',
 'urging',
 'carp',
 '##ool',
 'and',
 'tel',
 '##ew']

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True,max_length=512)

    labels = []
    for i, label in enumerate(examples[f"arg_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_rams = dataset_dict.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/7329 [00:00<?, ? examples/s]

Map:   0%|          | 0/924 [00:00<?, ? examples/s]

In [ ]:
tokenized_rams

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'event_tags', 'arg_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7329
    })
    test: Dataset({
        features: ['id', 'tokens', 'event_tags', 'arg_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 924
    })
})

In [ ]:
for i in range(7329):
  if len(tokenized_rams['train'][i]['input_ids']) > 500:
    print(len(tokenized_rams['train'][i]['input_ids']),i)

512 315
512 459
512 656
512 1268
512 2380
512 2939
512 3483
512 3959
512 4137
512 4477
512 4910
512 5196
512 5904


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:
labels = [label_list[i] for i in example["arg_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels,zero_division=1)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
encoder = LabelEncoder()
data['arg_tags_en'] = encoder.fit_transform(data['arg_tags'])

label2id = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
id2label = dict(zip(encoder.transform(encoder.classes_),encoder.classes_))

In [ ]:
label2id = {'B-arg01': 0,
 'B-arg02': 1,
 'B-arg03': 2,
 'B-arg04': 3,
 'B-arg05': 4,
 'I-arg01': 5,
 'I-arg02': 6,
 'I-arg03': 7,
 'I-arg04': 8,
 'I-arg05': 9,
 'O': 10}

In [ ]:
id2label = {0: 'B-arg01',
 1: 'B-arg02',
 2: 'B-arg03',
 3: 'B-arg04',
 4: 'B-arg05',
 5: 'I-arg01',
 6: 'I-arg02',
 7: 'I-arg03',
 8: 'I-arg04',
 9: 'I-arg05',
 10: 'O'}

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=11, id2label=id2label, label2id=label2id
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NLP_event_extraction/args_rams_model",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_rams["train"],
    eval_dataset=tokenized_rams["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.102500,0.162205,0.291076,0.200880,0.237710,0.964591
2,0.107200,0.162747,0.270928,0.234115,0.251180,0.962375
3,0.077000,0.179757,0.300362,0.243402,0.268898,0.963244
4,0.054800,0.189985,0.264948,0.251222,0.257903,0.961464
5,0.056800,0.217888,0.277262,0.233627,0.253581,0.962559
6,0.043600,0.228370,0.287965,0.223363,0.251583,0.963604
7,0.033800,0.243094,0.289426,0.234115,0.258849,0.963545


KeyboardInterrupt: 

In [ ]:
!pip install accelerate -U

In [ ]:
from transformers import AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP_event_extraction/args_rams_model/checkpoint-21987",max_length=512,padding=True)
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/NLP_event_extraction/args_rams_model/checkpoint-21987")

In [ ]:
for i in range(1000):
  idx = i
  c = dataset1[idx]['tokens']
  b = dataset1[idx]['event_tags']

  text = ' '.join(c)
  print(i)
  print(text)

  inputs = tokenizer(text,return_tensors="pt")
  # inputs = tokenize_and_align_labels(dataset1[idx])
  with torch.no_grad():
      logits = model(**inputs).logits


  predictions = torch.argmax(logits, dim=2)
  predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
  print(predictions)
  print(predicted_token_class)
  print(b)
  break

0
Transportation officials are urging carpool and teleworking as options to combat an expected flood of drivers on the road . ( Paul Duggan ) -- A Baltimore prosecutor accused a police detective of “ sabotaging ” investigations related to the death of Freddie Gray , accusing him of fabricating notes to suggest that the state ’s medical examiner believed the manner of death was an accident rather than a homicide . The heated exchange came in the chaotic sixth day of the trial of Baltimore Officer Caesar Goodson Jr. , who drove the police van in which Gray suffered a fatal spine injury in 2015 . ( Derek Hawkins and Lynh Bui )
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         3, 5, 5, 3, 5, 0, 0, 0, 0, 0, 0, 0,

KeyError: 'labels'